In [5]:
import sys
sys.path.insert(0, "../src/")
import jax.numpy as jnp
import numpy as np
import time
import compression_operator
from compression_operator import Top_k

In [6]:
def aCLAG_step(x_k, g_k, comm, oracle_container, compression_operator,
              trigger_beta, grad_k_prev, stepsize):
    x_k -= stepsize * g_k.mean(axis=0)
    n, d = g_k.shape
    triggered = np.array([False] * n)
    compressions = np.zeros(n,dtype=np.int)
    g_k_opt = np.zeros(g_k.shape)
    grad_k = oracle_container.compute_grad(x_k)
    for i in range(len(g_k)):
        for k in range(0, d, 5):
            compression_operator = Top_k(k)
            g_k_hat = g_k[i] + compression_operator.compress(grad_k[i] - g_k[i])
            trigger_rhs = trigger_beta * np.linalg.norm(
            grad_k[i] - grad_k_prev[i], ord=2, axis=1) ** 2
            trigger_lhs = np.linalg.norm(g_k_hat[i] - grad_k[i], ord=2) ** 2
            if trigger_lhs < trigger_rhs:
                compressions[i] = k
                g_k_opt[i] = g_k_hat
                triggered[i] = True
                break
        if not triggered[i]:
            compression_operator = Top_k(d)
            g_k_opt[i] = g_k[i] + compression_operator.compress(grad_k[i] - g_k[i])
            compressions[i] = d
            triggered[i] = True
    g_k = jnp.array(g_k_opt)
    comm += compressions.sum()
    return x_k, g_k, grad_k, comm

In [12]:
for i in range(2):
    for j in range(3):
        if j == 1:
            break
        print(i,j)

0 0
1 0


In [14]:
np.zeros(10,dtype=np.int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])